In [31]:
import sys
import numpy as np
import pandas as pd
from sklearn import preprocessing
from sklearn.svm import SVR
import matplotlib.pyplot as plt
%matplotlib inline

In [32]:
train = pd.read_csv("train.csv")
test = pd.read_csv("test.csv")

In [39]:
train.head(2)

,User_ID,Product_ID,Gender,Age,Occupation,City_Category,Stay_In_Current_City_Years,Marital_Status,Product_Category_1,Product_Category_2,Product_Category_3,Purchase
0,1000001,P00069042,F,0-17,10,A,2,0,3,NaN,NaN,8370
1,1000001,P00248942,F,0-17,10,A,2,0,1,6.0,14.0,15200


In [40]:
train.dtypes

User_ID                         int64
Product_ID                     object
Gender                         object
Age                            object
Occupation                      int64
City_Category                  object
Stay_In_Current_City_Years     object
Marital_Status                  int64
Product_Category_1              int64
Product_Category_2            float64
Product_Category_3            float64
Purchase                        int64
dtype: object

In [42]:
train['Product_Category_1'].unique()
train['Product_Category_2'].unique()
train['Product_Category_3'].unique()

array([nan, 14., 17.,  5.,  4., 16., 15.,  8.,  9., 13.,  6., 12.,  3.,
       18., 11., 10.])

In [43]:
train['Occupation'].unique()

array([10, 16, 15,  7, 20,  9,  1, 12, 17,  0,  3,  4, 11,  8, 19,  2, 18,
        5, 14, 13,  6], dtype=int64)

In [44]:
train.apply(lambda x: sum(x.isnull()))

User_ID                            0
Product_ID                         0
Gender                             0
Age                                0
Occupation                         0
City_Category                      0
Stay_In_Current_City_Years         0
Marital_Status                     0
Product_Category_1                 0
Product_Category_2            173638
Product_Category_3            383247
Purchase                           0
dtype: int64

In [45]:
df=pd.concat([train,test],ignore_index=True)

In [46]:
df.shape,train.shape,test.shape

((783667, 12), (550068, 12), (233599, 11))

In [47]:
df['Age']=df['Age'].map({'0-17':0, '55+':6, '26-35':2, '46-50':4, '51-55':5, '36-45':3, '18-25':1})

In [48]:
df['Age'].dtypes

dtype('int64')

In [49]:
df['Age'].unique()

array([0, 6, 2, 4, 5, 3, 1], dtype=int64)

In [50]:
df['Gender']=df['Gender'].map({'M':0,'F':1})
df['Gender'].unique()
df['City_Category']=df['City_Category'].map({'A':0,'B':1,'C':2})
df['City_Category'].unique()
df['Stay_In_Current_City_Years']=df['Stay_In_Current_City_Years'].map({'2':2, '4+':4, '3':3, '1':1, '0':0})
df['Stay_In_Current_City_Years'].unique()
df['Product_Category_2'].unique()
sum(df['Product_Category_2'].isnull())
df['Product_Category_2'].fillna(0,inplace=True)
df['Product_Category_3'].fillna(0,inplace=True)  

In [51]:
from sklearn.preprocessing import LabelEncoder
le1=LabelEncoder()
df['Product_ID']=le1.fit_transform(df['Product_ID'])

In [52]:
train=df.loc[df['Purchase'].isnull()==False]
test=df.loc[df['Purchase'].isnull()==True]

In [53]:
test.drop(['Purchase'],inplace=True,axis=1)

c:\users\ajitesh\appdata\local\programs\python\python36\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [106]:
from sklearn.tree import DecisionTreeRegressor
clf=DecisionTreeRegressor(random_state=0,max_depth=10)

In [107]:
target=['Purchase']
predictor=[x for x in train.columns if x not in target]
predictor

['Age',
 'City_Category',
 'Gender',
 'Marital_Status',
 'Occupation',
 'Product_Category_1',
 'Product_Category_2',
 'Product_Category_3',
 'Product_ID',
 'Stay_In_Current_City_Years',
 'User_ID']

In [108]:
clf.fit(train[predictor],train[target])
predictions = clf.predict(test[predictor])

In [109]:
from sklearn import cross_validation
kf_total = cross_validation.KFold(len(train), n_folds=10,shuffle=True, random_state=24)

In [110]:
score=cross_validation.cross_val_score(clf, train[predictor], train[target], cv=kf_total, n_jobs=1)

In [111]:
score.mean() 

0.6856697348353724

In [112]:

out_df = pd.DataFrame({"User_ID":test['User_ID']})
out_df["Product_ID"] = test['Product_ID']
out_df["Purchase"] = predictions
out_df.to_csv("sub2.csv", index=False)

In [113]:
out_df.head()

,User_ID,Product_ID,Purchase
550068,1000004,1216,15617.073846
550069,1000009,1063,11565.576531
550070,1000010,2799,6040.021705
550071,1000010,1379,2463.914573
550072,1000011,535,2585.590829
